In [ ]:
from glob import glob
import os

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import caffe
import pandas as pd

In [ ]:
# set display defaults
plt.rcParams['figure.figsize'] = (10, 10)        # large images
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels
plt.rcParams['image.cmap'] = 'gray'  # use grayscale output rather than a (potentially misleading) color heatmap

In [ ]:
def get_split(split):
    filename = './development_kit/data/%s.txt' % split
    if not os.path.exists(filename):
        raise IOError('Split data file not found: %s' % split)
    return filename

In [ ]:
def get_pred(base, source):
    return os.path.join(base, 'top_5_predictions.{}.csv'.format(source))

In [ ]:
def vis_square(data):
    """Take an array of shape (n, height, width) or (n, height, width, 3)
       and visualize each (height, width) thing in a grid of size approx. sqrt(n) by sqrt(n)"""
    
    # normalize data for display
    data = (data - data.min()) / (data.max() - data.min())
    
    # force the number of filters to be square
    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = (((0, n ** 2 - data.shape[0]),
               (0, 1), (0, 1))                 # add some space between filters
               + ((0, 0),) * (data.ndim - 3))  # don't pad the last dimension (if there is one)
    data = np.pad(data, padding, mode='constant', constant_values=1)  # pad with ones (white)
    
    # tile the filters into an image
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3) + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    
    plt.imshow(data); plt.axis('off')

In [ ]:
cat = pd.read_csv('development_kit/data/categories.txt', sep=' ', header=None, names=['category', 'label'])

In [ ]:
base_path = '/home/mossing/caffemodels_for_alex/*'

In [ ]:
model_dirs = glob(base_path)

# Model Performance for different Architectures

In [ ]:
for split in ['train', 'val']:
    data = pd.read_csv(get_split(split), sep=' ', header=None, names=['image', 'true_label'])
    for q, model_dir in enumerate(model_dirs):
        name = model_dir.split('/')[-1]
        print 'Model name: {}'.format(name)
#         print 'Split: {}'.format(split)
        tmp = ['{}label{}'.format(name, i) for i in [1,2,3,4,5]]
        pred = pd.read_csv(get_pred(model_dir, split), header=True, names=(['image'] + tmp))
        data = pd.merge(data, pred, on='image')

        top1 = (data['true_label'] == data['{}label1'.format(name)]).mean()
        print 'Split: {:6}, Top 1 accuracy: {:.2f} %'.format(split, top1 * 100)

In [ ]:
data.head()

# Plot the first layer filters

In [ ]:
plt.figure(figsize=(10, 10))
for i, model_dir in enumerate(model_dirs):
    proto_fn = glob(model_dir + '/*proto*')[0]
    model_fn = glob(model_dir + '/snapshot/*50000.caffemodel')[0]

    net = caffe.Net(proto_fn, model_fn, caffe.TEST)

    plt.subplot(2, 2, i+1)
    plt.title(model_dir.split('/')[-1])
    filters = net.params['conv0'][0].data
    vis_square(filters.transpose(0, 2, 3, 1))

# Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cm = confusion_matrix(data['true_label'], data['deeperlabel1'])

In [ ]:
cat['val_correct'] = np.diag(cm) / (1. * cm.shape[0])

In [ ]:
from scipy.cluster import hierarchy

In [ ]:
cat.sort('val_correct')